In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

# Config
IMAGE_FILE = "../DataSet/Smaller Velocity Dataset/Town06_500s-3_images_Smaller.npy"
VELOCITY_FILE = "../DataSet/Smaller Velocity Dataset/Town06_500s-3_velocity_Smaller.npy"
MODEL_PATH = "../Models/velocity_regressor.pth"
BATCH_SIZE = 32

# Dataset
class VelocityDataset(Dataset):
    def __init__(self, image_path, velocity_path, transform=None):
        self.images = np.load(image_path)
        self.velocities = np.load(velocity_path)

        # If channels-first, switch to HWC for PIL
        if self.images.ndim == 4 and self.images.shape[1] == 3:
            self.images = np.transpose(self.images, (0, 2, 3, 1))

        # If normalized to [0,1], convert to uint8
        if self.images.max() <= 1.0:
            self.images = (self.images * 255).astype(np.uint8)

        self.transform = transform

    def __len__(self):
        return len(self.velocities)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        velocity = self.velocities[idx]

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor([velocity], dtype=torch.float32)

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Model definition
class VelocityRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=False)
        base.fc = nn.Linear(base.fc.in_features, 1)
        self.model = base

    def forward(self, x):
        return self.model(x)

# Load data
dataset = VelocityDataset(IMAGE_FILE, VELOCITY_FILE, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VelocityRegressor().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# Evaluate
mse_criterion = nn.MSELoss()
mae_criterion = nn.L1Loss()

all_preds = []
all_targets = []

with torch.no_grad():
    for images, targets in dataloader:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        all_preds.append(outputs.cpu())
        all_targets.append(targets.cpu())

preds = torch.cat(all_preds, dim=0)
targets = torch.cat(all_targets, dim=0)

mse = mse_criterion(preds, targets).item()
mae = mae_criterion(preds, targets).item()

print("Evaluation complete:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")


c:\Users\aadav\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aadav\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Evaluation complete:
🔸 Mean Squared Error (MSE): 0.1126
🔸 Mean Absolute Error (MAE): 0.2521
